In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torch.nn.functional as F


import torchvision
from torchvision.models import resnet18, ResNet18_Weights, vgg16, VGG16_Weights
from torchvision import datasets
from torchvision import transforms

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt


import os
import sys
import copy
from torchsummary import summary


# from google.colab import drive
# drive.mount('/content/drive')
# #Code folder path
# %cd /content/drive/My Drive/ECE591_DL_CL_PROJECT/

#device config
torch.cuda.set_device(1)#Select GPU device 1
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Get Packages
sys.path.append('./Tools')

#Network architectures
from ResNet18_CIFAR10 import ResNet18_CIFAR10_Model
from ResNet18_TINYIMAGENET import ResNet18_TINYIMAGENET_Model
from VGG16_CIFAR10 import VGG16_CIFAR10_Model
from VGG16_TINYIMAGENET import VGG16_TINYIMAGENET_Model

#Non Corrupted DataLoaders
from CIFAR10_LOADER import CIFAR10DataLoader
from TINYIMAGENET_LOADER import TINYIMAGENETDataLoader

#Non Corrupted DataLoaders
from CIFAR10C_LOADER import CIFAR10C_DataLoader #Each Corruption is a entry in the dictionary
from TINYIMAGENET_C_LOADER import TinyImagenetC_DataLoader #Each Corruption is a entry in the dictionary

#Error Metrics
from ErrorMetrics import Evaluate_Model_TOP1
from PruningWithAdversarialExamples import iterative_pruning


# Load Datasets

In [2]:
from unittest import loader

Times=5
trainloader_CIFAR10, testloader_CIFAR10=CIFAR10DataLoader(batch_size=512*Times) #Load Data for CIFAR10
loader_CIFAR10C=CIFAR10C_DataLoader(batch_size=512*Times) #Load Data for CIFAR10C

Times=1
trainloader_TINYIMAGENET, testloader_TINYIMAGENET=TINYIMAGENETDataLoader(NPZ=True,batch_size=512*Times) #Load Data for TINYIMAGENET
loader_TinyImagenetC=TinyImagenetC_DataLoader(NPZ=True,batch_size=512*Times) #Load Data for TINYIMAGENETC

MeanCIFAR10=(0.491, 0.482, 0.446)
StdCIFAR10=(0.247, 0.243, 0.261)
MeanImagenet=(0.485, 0.456, 0.406)
StdImagenet=(0.229, 0.224, 0.225)

Files already downloaded and verified
Files already downloaded and verified


# CIFAR 10

## Check effect of level of adversarial examples (epsilon)

In [3]:
#Network weights
ResNet18_CIFAR10_Weights= torch.load("./Networks/ResNet18_CIFAR10_V0_Adversarial_ep_3p.pt")
ResNet18_CIFAR10_0P=ResNet18_CIFAR10_Model()
ResNet18_CIFAR10_0P.load_state_dict(ResNet18_CIFAR10_Weights)

model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

epsilon=0.03
mean=MeanCIFAR10
std=StdCIFAR10


SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR10_V0_Adversarial_ep_3p_01PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with CIFAR10 Dataset with Adversarial training (Before and During Pruning) with epsilon=0.03\n"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                         epsilon, mean, std, lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.7283, Accuracy: 40844/50000 (81.69%)
Test set: Average loss: 0.7499, Accuracy: 7780/10000 (77.80%)

--Testing Corruptions--


100%|██████████| 19/19 [02:47<00:00,  8.80s/it]



Avg Corruption Accuracy: 70.88%
---------- Pruning Iteration:  1 / 44


Loss=1.33950936794281 Batch_id=19 Accuracy=48.74: 100%|██████████| 20/20 [00:28<00:00,  1.44s/it]  
Loss=1.3659802675247192 Batch_id=19 Accuracy=48.47: 100%|██████████| 20/20 [00:29<00:00,  1.48s/it]
Loss=1.3501019477844238 Batch_id=19 Accuracy=48.92: 100%|██████████| 20/20 [00:31<00:00,  1.60s/it]
Loss=1.2474002838134766 Batch_id=19 Accuracy=49.23: 100%|██████████| 20/20 [00:29<00:00,  1.46s/it]
Loss=1.247696876525879 Batch_id=19 Accuracy=49.01: 100%|██████████| 20/20 [00:29<00:00,  1.47s/it] 
Loss=1.3141919374465942 Batch_id=19 Accuracy=49.32: 100%|██████████| 20/20 [00:29<00:00,  1.48s/it]
Loss=1.3049957752227783 Batch_id=19 Accuracy=49.50: 100%|██████████| 20/20 [00:29<00:00,  1.47s/it]


Epoch 00007: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00007: reducing learning rate of group 0 to 5.0000e-04.


Loss=1.5279853343963623 Batch_id=19 Accuracy=42.57: 100%|██████████| 20/20 [00:25<00:00,  1.29s/it]
Loss=1.4354240894317627 Batch_id=19 Accuracy=42.67: 100%|██████████| 20/20 [00:25<00:00,  1.29s/it]
Loss=1.4893906116485596 Batch_id=19 Accuracy=42.63: 100%|██████████| 20/20 [00:25<00:00,  1.29s/it]


Test set: Average loss: 0.9456, Accuracy: 7021/10000 (70.21%)

--Testing Corruptions--


100%|██████████| 19/19 [03:23<00:00, 10.69s/it]


Avg Corruption Accuracy: 64.26%
Global Sparsity: 98.98% 



In [4]:
#Network weights
ResNet18_CIFAR10_Weights= torch.load("./Networks/ResNet18_CIFAR10_V0_Adversarial_ep_1p.pt")
ResNet18_CIFAR10_0P=ResNet18_CIFAR10_Model()
ResNet18_CIFAR10_0P.load_state_dict(ResNet18_CIFAR10_Weights)

model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

epsilon=0.01
mean=MeanCIFAR10
std=StdCIFAR10


SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR10_V0_Adversarial_ep_1p_01PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with CIFAR10 Dataset with Adversarial training (Before and During Pruning) with epsilon=0.01\n"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                         epsilon, mean, std, lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.3384, Accuracy: 45442/50000 (90.88%)
Test set: Average loss: 0.4362, Accuracy: 8528/10000 (85.28%)

--Testing Corruptions--


100%|██████████| 19/19 [03:21<00:00, 10.59s/it]



Avg Corruption Accuracy: 77.23%
---------- Pruning Iteration:  1 / 44


Loss=0.7307682633399963 Batch_id=19 Accuracy=72.37: 100%|██████████| 20/20 [00:26<00:00,  1.32s/it]
Loss=0.6831562519073486 Batch_id=19 Accuracy=72.32: 100%|██████████| 20/20 [00:26<00:00,  1.32s/it]
Loss=0.669631838798523 Batch_id=19 Accuracy=72.59: 100%|██████████| 20/20 [00:26<00:00,  1.32s/it] 
Loss=0.7013301849365234 Batch_id=19 Accuracy=73.11: 100%|██████████| 20/20 [00:26<00:00,  1.33s/it]
Loss=0.7148966789245605 Batch_id=19 Accuracy=73.18: 100%|██████████| 20/20 [00:26<00:00,  1.32s/it]
Loss=0.6963679790496826 Batch_id=19 Accuracy=73.43: 100%|██████████| 20/20 [00:26<00:00,  1.32s/it]


Epoch 00006: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00009: reducing learning rate of group 0 to 5.0000e-04.


Loss=0.8533664345741272 Batch_id=19 Accuracy=68.11: 100%|██████████| 20/20 [00:28<00:00,  1.42s/it]


Test set: Average loss: 0.5237, Accuracy: 8213/10000 (82.13%)

--Testing Corruptions--


100%|██████████| 19/19 [03:58<00:00, 12.55s/it]


Avg Corruption Accuracy: 74.39%
Global Sparsity: 98.98% 



In [6]:
#Network weights
ResNet18_CIFAR10_Weights= torch.load("./Networks/ResNet18_CIFAR10_V0_Adversarial_ep_05p.pt")
ResNet18_CIFAR10_0P=ResNet18_CIFAR10_Model()
ResNet18_CIFAR10_0P.load_state_dict(ResNet18_CIFAR10_Weights)

model=copy.deepcopy(ResNet18_CIFAR10_0P)
TrainLoader=trainloader_CIFAR10
TestLoader=testloader_CIFAR10
TestCLoader=loader_CIFAR10C
PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

epsilon=0.005
mean=MeanCIFAR10
std=StdCIFAR10


SaveTrainStepsName="Progress/PruningIterations_ResNet18_CIFAR10_V0_Adversarial_ep_05p_01PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with CIFAR10 Dataset with Adversarial training (Before and During Pruning) with epsilon=0.005\n"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                         epsilon, mean, std, lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.0685, Accuracy: 49289/50000 (98.58%)
Test set: Average loss: 0.3598, Accuracy: 8826/10000 (88.26%)

--Testing Corruptions--


100%|██████████| 19/19 [04:29<00:00, 14.16s/it]



Avg Corruption Accuracy: 80.12%
---------- Pruning Iteration:  1 / 44


Loss=0.21204030513763428 Batch_id=19 Accuracy=93.26: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it]
Loss=0.19619117677211761 Batch_id=19 Accuracy=93.26: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it]
Loss=0.2030138522386551 Batch_id=19 Accuracy=93.13: 100%|██████████| 20/20 [00:29<00:00,  1.50s/it] 
Loss=0.19069232046604156 Batch_id=19 Accuracy=93.68: 100%|██████████| 20/20 [00:29<00:00,  1.50s/it]
Loss=0.17219491302967072 Batch_id=19 Accuracy=93.92: 100%|██████████| 20/20 [00:29<00:00,  1.50s/it]
Loss=0.1786758005619049 Batch_id=19 Accuracy=93.97: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it] 
Loss=0.15528349578380585 Batch_id=19 Accuracy=94.18: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it]
Loss=0.17496387660503387 Batch_id=19 Accuracy=94.62: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it]
Loss=0.1458907425403595 Batch_id=19 Accuracy=94.30: 100%|██████████| 20/20 [00:30<00:00,  1.51s/it] 
Loss=0.16781537234783173 Batch_id=19 Accuracy=94.49: 100%|██████████| 20/20 [00:29<00:00,  

Test set: Average loss: 0.3897, Accuracy: 8771/10000 (87.71%)

--Testing Corruptions--


...

Test set: Average loss: 0.4272, Accuracy: 8542/10000 (85.42%)

--Skipping Acc Comp For Corrupted Data--
Global Sparsity: 98.88% 

---------- Pruning Iteration:  44 / 44


Loss=0.6683946847915649 Batch_id=19 Accuracy=73.49: 100%|██████████| 20/20 [00:30<00:00,  1.54s/it]
Loss=0.6359760761260986 Batch_id=19 Accuracy=76.04: 100%|██████████| 20/20 [00:31<00:00,  1.56s/it]
Loss=0.6261833310127258 Batch_id=19 Accuracy=76.61: 100%|██████████| 20/20 [00:31<00:00,  1.56s/it]
Loss=0.6023257374763489 Batch_id=19 Accuracy=77.06: 100%|██████████| 20/20 [00:30<00:00,  1.55s/it]
Loss=0.6482033133506775 Batch_id=19 Accuracy=77.39: 100%|██████████| 20/20 [00:31<00:00,  1.55s/it]
Loss=0.6212013959884644 Batch_id=19 Accuracy=77.87: 100%|██████████| 20/20 [00:31<00:00,  1.56s/it]
Loss=0.5873228311538696 Batch_id=19 Accuracy=77.90: 100%|██████████| 20/20 [00:31<00:00,  1.55s/it]
Loss=0.6127562522888184 Batch_id=19 Accuracy=78.01: 100%|██████████| 20/20 [00:31<00:00,  1.56s/it]
Loss=0.560832142829895 Batch_id=19 Accuracy=77.89: 100%|██████████| 20/20 [00:30<00:00,  1.55s/it] 
Loss=0.557945966720581 Batch_id=19 Accuracy=78.38: 100%|██████████| 20/20 [00:30<00:00,  1.55s/it] 


Test set: Average loss: 0.4350, Accuracy: 8532/10000 (85.32%)

--Testing Corruptions--


100%|██████████| 19/19 [04:59<00:00, 15.75s/it]


Avg Corruption Accuracy: 75.90%
Global Sparsity: 98.98% 



# Imagenet

In [7]:
#Choose a corruption level because simulation every level is too much
TinyImagenetC_CorruptionClasses=list(loader_TinyImagenetC.keys())
TinyImagenetC_CorruptionLevels=list(loader_TinyImagenetC[TinyImagenetC_CorruptionClasses[0]].keys()) #['1', '2', '3', '4', '5']

#Choose a level
clevel='1'
TiniyImageNetCLevel_Loader={}#Filtered for a specifig level of corruption
for cname in loader_TinyImagenetC.keys():
    TiniyImageNetCLevel_Loader[cname]=loader_TinyImagenetC[cname][clevel]

In [8]:
#Network weights
ResNet18_TINYIMAGENET_Weights= torch.load("./Networks/ResNet18_TinyImagenet_V0_Adversarial_ep_05p.pt")
ResNet18_TINYIMAGENET_0P=ResNet18_TINYIMAGENET_Model()
ResNet18_TINYIMAGENET_0P.load_state_dict(ResNet18_TINYIMAGENET_Weights)

model=copy.deepcopy(ResNet18_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader

PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

epsilon=0.005
mean=MeanImagenet
std=StdImagenet


SaveTrainStepsName="Progress/PruningIterations_ResNet18_TiniyImageNet_V0_Adversarial_ep_05p_01PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with TinyImagenet Dataset with Adversarial training (Before and During Pruning) with epsilon=0.005\n"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                         epsilon, mean, std, lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.5325, Accuracy: 88266/100000 (88.27%)
Test set: Average loss: 1.7649, Accuracy: 2926/5000 (58.52%)

--Testing Corruptions--


100%|██████████| 19/19 [03:52<00:00, 12.26s/it]



Avg Corruption Accuracy: 30.25%
---------- Pruning Iteration:  1 / 44


Loss=1.568792700767517 Batch_id=195 Accuracy=65.90: 100%|██████████| 196/196 [02:34<00:00,  1.27it/s] 
Loss=1.3202171325683594 Batch_id=195 Accuracy=65.31: 100%|██████████| 196/196 [02:34<00:00,  1.27it/s]
Loss=1.1180466413497925 Batch_id=195 Accuracy=65.94: 100%|██████████| 196/196 [02:34<00:00,  1.27it/s]
Loss=1.3495702743530273 Batch_id=195 Accuracy=66.26: 100%|██████████| 196/196 [02:34<00:00,  1.27it/s]
Loss=1.3116905689239502 Batch_id=195 Accuracy=66.71: 100%|██████████| 196/196 [02:34<00:00,  1.27it/s]
Loss=1.348490595817566 Batch_id=195 Accuracy=66.94: 100%|██████████| 196/196 [02:34<00:00,  1.27it/s] 


Epoch 00006: reducing learning rate of group 0 to 5.0000e-04.


...

Epoch 00008: reducing learning rate of group 0 to 2.5000e-05.


Loss=2.151503801345825 Batch_id=195 Accuracy=48.93: 100%|██████████| 196/196 [02:36<00:00,  1.25it/s] 
Loss=2.053088665008545 Batch_id=195 Accuracy=48.92: 100%|██████████| 196/196 [02:36<00:00,  1.25it/s] 


Test set: Average loss: 1.9921, Accuracy: 2587/5000 (51.74%)

--Testing Corruptions--


100%|██████████| 19/19 [05:07<00:00, 16.20s/it]


Avg Corruption Accuracy: 24.60%
Global Sparsity: 98.13% 



In [11]:
#Network weights
ResNet18_TINYIMAGENET_Weights= torch.load("./Networks/ResNet18_TinyImagenet_V0_Adversarial_ep_1p.pt")
ResNet18_TINYIMAGENET_0P=ResNet18_TINYIMAGENET_Model()
ResNet18_TINYIMAGENET_0P.load_state_dict(ResNet18_TINYIMAGENET_Weights)

model=copy.deepcopy(ResNet18_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader

PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

epsilon=0.01
mean=MeanImagenet
std=StdImagenet


SaveTrainStepsName="Progress/PruningIterations_ResNet18_TiniyImageNet_V0_Adversarial_ep_01p_01PR_10Epochs_L2_0005"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with TinyImagenet Dataset with Adversarial training (Before and During Pruning) with epsilon=0.01\n"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                         epsilon, mean, std, lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)

--Get statistics before pruning--
Train set: Average loss: 0.8718, Accuracy: 80089/100000 (80.09%)


In [ ]:
#Network weights
ResNet18_TINYIMAGENET_Weights= torch.load("./Networks/ResNet18_TinyImagenet_V0_Adversarial_ep_1p.pt")
ResNet18_TINYIMAGENET_0P=ResNet18_TINYIMAGENET_Model()
ResNet18_TINYIMAGENET_0P.load_state_dict(ResNet18_TINYIMAGENET_Weights)

model=copy.deepcopy(ResNet18_TINYIMAGENET_0P)
TrainLoader=trainloader_TINYIMAGENET
TestLoader=testloader_TINYIMAGENET
TestCLoader=TiniyImageNetCLevel_Loader

PruningRate=0.1
NumEpochsEachPruneRetrain=10
MaximumPruneLevel=0.01
lambda_l1=0
lambda_l2=0.0005

epsilon=0.03
mean=MeanImagenet
std=StdImagenet


SaveTrainStepsName="Progress/PruningIterations_ResNet18_TiniyImageNet_V0_Adversarial_ep_03p_01PR_10Epochs_L2_0001"


ReadMe="This model is running pruning with the standard cost function, and the non corrupted dataset, a prunning rate of 0.1 and 10 epoch of retraining\
L2 Regularization of 0.0005 is used\n\nOriginal Model: ResNet18 Trained with TinyImagenet Dataset with Adversarial training (Before and During Pruning) with epsilon=0.03\n"

test_acc, Avg_Corrup_Acc_List, Acc_Corruptions, RemainingWeights_List, Sparcity_List=iterative_pruning(model,device, TrainLoader, TestLoader, TestCLoader, PruningRate, SaveTrainStepsName,ReadMe,
                                                                                                         epsilon, mean, std, lambda_l1, lambda_l2, NumEpochsEachPruneRetrain,MaximumPruneLevel)